In [1]:
!pip install -q pyautogen
!pip install -q python-dotenv
!pip install -q pyautogen[anthropic]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 5.2 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

In [3]:
import os
from autogen import ConversableAgent

In [4]:
import os
os.environ["OPENAI_API_KEY"]=userdata.get("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"]=userdata.get("ANTHROPIC_API_KEY")


In [ ]:
config_list_gpt={
    {
        model:'gpt-4o-mini',
        "api_key":user.userdata('OPENAI_API_KEY')
    }

}

In [ ]:
config_list_gpt={
    {
        model:'claude-3-5-sonnet-20240620',
        "api_key":user.userdata('ANTHROPIC_API_KEY'),
        "api_type":"anthropic"
    }

}

In [ ]:
pizza_agent = ConversableAgent(
    name="pizza_lover",
    system_message="You are a person who loves pizza and wants to spread its deliciousness around the world. Speak passionately about the allure of pizza.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

In [ ]:
sushi_agent = ConversableAgent(
    name="sushi_lover",
    system_message="You are a person who loves sushi and wants to spread its deliciousness around the world. Speak passionately about the allure of sushi.",
    llm_config={"config_list": config_list_gpt},
    human_input_mode="NEVER",
)

In [ ]:
judge_agent = ConversableAgent(
    name="judge_Agent",
    system_message="You are acting as the ultimate facilitator. Your job is to guide the debate between the two and declare a winner based on who makes the most convincing argument. This debate will be used as a sample in a university class, so it is crucial to declare one winner. Once a clear conclusion is reached, you must declare 'That's enough!' and announce the winner. The debate cannot end without this phrase, so make sure to include it.",
    llm_config={"config_list": config_list_gpt},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "That's enough!" in msg["content"],

In [ ]:
pizza_agent.description = "The ultimate pizza fan"
sushi_agent.description = "The ultimate sushi fan"
judge_agent.description = "The facilitator who decides the debate winner"


In [ ]:

from autogen import GroupChat

In [ ]:
group_chat = GroupChat(
    agents=[pizza_agent, sushi_agent,judge_agent],
    messages=[],
    send_introductions=True,
    speaker_selection_method = "auto",
    max_round = 5
)

In [ ]:
from autogen import GroupChatManager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4o-mini",
                                 "api_key": os.environ["OPENAI_API_KEY"]}]},

)

In [ ]:
chat_result = judge_agent.initiate_chat(
    group_chat_manager,
    message="This debate will be used as a sample in a university class. A winner must be decided. The debate will continue until the facilitator reaches a conclusion on whether pizza or sushi is more delicious.",
    summary_method="reflection_with_llm",
)